In [11]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
print(torch.version.cuda)

12.4


In [12]:
# Define node features (e.g., weights or attributes of stations)
# Assume you have 4 stations (nodes), each with a feature vector of size 2
node_features = torch.tensor([[1.0, 2.0],
                              [2.0, 3.0], 
                              [3.0, 4.0], 
                              [4.0, 5.0]], dtype=torch.float)

# Define edge connections (source and target nodes)
# For example, edges between stations: 0 -> 1, 1 -> 2, etc.
edge_index = torch.tensor([[0, 1, 2, 3],
                           [1, 2, 3, 0]], dtype=torch.long)  # Shape: [2, num_edges]

# Define edge weights (e.g., distances or connection strengths)
edge_weights = torch.tensor([1.5, 2.5, 3.5, 4.5], dtype=torch.float)

# Create the graph data object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights)

print(data)

Data(x=[4, 2], edge_index=[2, 4], edge_attr=[4])


In [13]:
# Create multiple graphs (one for each time interval)
graphs = []
for t in range(10):  # Example: Create 10 graphs for different intervals
    node_features = torch.rand((4, 2))  # Random node features for this example
    edge_index = torch.tensor([[0, 1, 2, 3],
                                [1, 2, 3, 0]], dtype=torch.long)
    edge_weights = torch.rand(4)       # Random edge weights for this example
    graphs.append(Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights))

# Create a DataLoader for batching graphs during training
loader = DataLoader(graphs, batch_size=2)  # Batch size of 2 graphs

for batch in loader:
    print(batch)

DataBatch(x=[8, 2], edge_index=[2, 8], edge_attr=[8], batch=[8], ptr=[3])
DataBatch(x=[8, 2], edge_index=[2, 8], edge_attr=[8], batch=[8], ptr=[3])
DataBatch(x=[8, 2], edge_index=[2, 8], edge_attr=[8], batch=[8], ptr=[3])
DataBatch(x=[8, 2], edge_index=[2, 8], edge_attr=[8], batch=[8], ptr=[3])
DataBatch(x=[8, 2], edge_index=[2, 8], edge_attr=[8], batch=[8], ptr=[3])


In [14]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels=2, out_channels=16)   # Input size matches node feature size
        self.conv2 = GCNConv(in_channels=16, out_channels=8)   # Hidden layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First GCN layer with ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        
        # Second GCN layer (output layer)
        x = self.conv2(x, edge_index)
        return x


In [15]:
from torch.optim import Adam

# Initialize model and optimizer
model = GNN()
optimizer = Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(100):   # Number of epochs
    model.train()
    total_loss = 0
    
    for batch in loader:
        optimizer.zero_grad()
        
        # Forward pass through the model
        out = model(batch)
        
        # Example loss function (e.g., Mean Squared Error for regression tasks)
        target = torch.rand((batch.num_nodes, 8))   # Random target for demonstration
        loss = F.mse_loss(out, target)
        
        loss.backward()      # Backpropagation
        optimizer.step()     # Update parameters
        
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {total_loss:.4f}')


Epoch 1, Loss: 1.1365
Epoch 2, Loss: 0.6578
Epoch 3, Loss: 0.4567
Epoch 4, Loss: 0.5721
Epoch 5, Loss: 0.4968
Epoch 6, Loss: 0.4519
Epoch 7, Loss: 0.5042
Epoch 8, Loss: 0.4478
Epoch 9, Loss: 0.4659
Epoch 10, Loss: 0.4702
Epoch 11, Loss: 0.4523
Epoch 12, Loss: 0.4578
Epoch 13, Loss: 0.4548
Epoch 14, Loss: 0.4610
Epoch 15, Loss: 0.4520
Epoch 16, Loss: 0.4758
Epoch 17, Loss: 0.4104
Epoch 18, Loss: 0.3936
Epoch 19, Loss: 0.4456
Epoch 20, Loss: 0.4572
Epoch 21, Loss: 0.4601
Epoch 22, Loss: 0.4695
Epoch 23, Loss: 0.4533
Epoch 24, Loss: 0.4467
Epoch 25, Loss: 0.4174
Epoch 26, Loss: 0.4291
Epoch 27, Loss: 0.3914
Epoch 28, Loss: 0.4209
Epoch 29, Loss: 0.4283
Epoch 30, Loss: 0.4218
Epoch 31, Loss: 0.4206
Epoch 32, Loss: 0.4498
Epoch 33, Loss: 0.4192
Epoch 34, Loss: 0.4252
Epoch 35, Loss: 0.3920
Epoch 36, Loss: 0.4172
Epoch 37, Loss: 0.4724
Epoch 38, Loss: 0.4529
Epoch 39, Loss: 0.4036
Epoch 40, Loss: 0.4118
Epoch 41, Loss: 0.4133
Epoch 42, Loss: 0.3915
Epoch 43, Loss: 0.4199
Epoch 44, Loss: 0.38